In [1]:
import pandas as pd
import numpy as np

***Data Cleaning and Exploration***

In [2]:
epi = pd.read_csv('./data/dataframes/epi_wrangled.csv')

In [3]:
epi

,year,country_name,Unsafe sanitation,Sanitation & Drinking Water,Terrestrial biome protection (national weights),PM2.5 Exposure,Unsafe drinking water,Water Resources,Ecosystem Vitality,Agriculture,...,Trend in CO2 Emissions per KWH,Climate & Energy,Fisheries,Marine Protected Areas,Fish Stock Status,Trend in Carbon Intensity,Coastal Shelf Fishing Pressure,Change of Trend in Carbon Intensity,Climate,GDP
0,2002,Afghanistan,2.74,1.37,1.65,96.15,0.00,0.00,12.62,50.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.128821e+09
1,2003,Afghanistan,2.88,1.44,1.65,95.16,0.00,0.00,12.62,50.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.583644e+09
2,2004,Afghanistan,3.01,1.51,1.65,98.83,0.00,0.00,12.62,50.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.285466e+09
3,2005,Afghanistan,3.15,2.20,1.65,96.38,1.25,0.00,12.62,50.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.275074e+09
4,2006,Afghanistan,3.30,2.98,1.65,96.21,2.66,0.00,12.62,50.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.057598e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3461,2016,Brazil,77.80,84.42,99.09,100.00,93.29,76.23,70.67,72.98,...,70.837406,56.41,65.40,95.27,65.40,57.006664,NaN,NaN,NaN,2.939094e+12
3462,2017,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.017716e+12
3463,2018,Brazil,43.46,45.91,99.88,100.00,48.37,81.08,56.21,58.83,...,NaN,33.24,81.42,94.88,62.84,NaN,NaN,NaN,NaN,3.131946e+12
3464,2019,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.229055e+12


In [4]:
epi['country_name'].value_counts()

Turkmenistan         19
Albania              19
Fiji                 19
Brunei Darussalam    19
Moldova              19
                     ..
San Marino            1
Réunion               1
American Samoa        1
Isle of Man           1
Greenland             1
Name: country_name, Length: 226, dtype: int64

In [5]:
def impute_values(country_name):
    country = epi[epi['country_name'] == country_name].copy()
    for col in country.columns:
        country[col] = country[col].map(lambda x: np.nan if x == 'None' else x)
        country[col].interpolate(method='linear',order=1,inplace=True, axis=0)
    return country
        

In [6]:
epi_new = pd.DataFrame()

for country_name in  set(epi['country_name'].values):
    country_df = impute_values(country_name)
    epi_new = pd.concat([epi_new, country_df], ignore_index = True)

In [7]:
epi_new.head()

,year,country_name,Unsafe sanitation,Sanitation & Drinking Water,Terrestrial biome protection (national weights),PM2.5 Exposure,Unsafe drinking water,Water Resources,Ecosystem Vitality,Agriculture,...,Trend in CO2 Emissions per KWH,Climate & Energy,Fisheries,Marine Protected Areas,Fish Stock Status,Trend in Carbon Intensity,Coastal Shelf Fishing Pressure,Change of Trend in Carbon Intensity,Climate,GDP
0,2012,Greenland,100.00,100.00,100.0,100.0,100.00,NaN,NaN,NaN,...,NaN,NaN,23.50,NaN,22.04,NaN,24.96,NaN,NaN,2.408666e+09
1,2002,Argentina,52.15,59.92,29.1,100.0,67.68,11.75,24.36,92.0,...,11.47,16.79,14.02,64.53,12.37,13.6,15.68,22.06,16.79,9.772400e+10
2,2003,Argentina,53.34,61.33,29.1,100.0,69.33,11.75,24.82,92.0,...,11.47,16.79,18.65,64.53,21.15,13.6,16.15,22.06,16.79,1.280000e+11
3,2004,Argentina,54.58,62.81,29.1,100.0,71.04,11.75,24.64,92.0,...,11.47,16.79,16.81,64.53,16.67,13.6,16.95,22.06,16.79,1.650000e+11
4,2005,Argentina,55.87,64.35,29.1,100.0,72.82,11.75,24.77,96.0,...,11.47,16.79,16.19,64.53,15.81,13.6,16.57,22.06,16.79,1.990000e+11


In [8]:
epi_new.shape

(3466, 36)

In [9]:
nulls = epi_new.isnull().sum().sort_values(ascending =False)
nulls[nulls > 0]

Critical Habitat Protection                        1962
Climate                                             998
Trend in CO2 Emissions per KWH                      995
Change of Trend in Carbon Intensity                 994
Marine Protected Areas                              893
Trend in Carbon Intensity                           888
Change in Forest Cover                              859
Coastal Shelf Fishing Pressure                      854
Fisheries                                           851
Fish Stock Status                                   850
Climate & Energy                                    840
Forests                                             701
Agricultural Subsidies                              505
Environmental Performance Index                      79
Ecosystem Vitality                                   79
Water Resources                                      55
Wastewater Treatment                                 55
Environmental Health                            